<center><h1><font size="6">DataKind/UChicago CDAC DataDive Event Notebook</font></h1></center>

# Table of Contents

*   <a href='#1'>Importing Packages</a>
*   <a href='#2'>Importing NY Shapefile</a>
*   <a href='#3'>ACS Database</a>
*   <a href='#4'>FCC Database</a>
*   <a href='#5'>Ookla Database</a>
*   <a href='#6'>Using Spatial Join</a>
*   <a href='#7'>Joining with Centroids</a>
*   <a href='#8'>Illinois Datasets</a>
*   <a href='#9'>Florida Datasets</a>



# <a id='1'>Importing Packages</a>

In [1]:
# for Google Colab only
# from google.colab import drive
# drive.mount('/content/drive')

# Important library for many geopython libraries
# !apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
# !apt install python3-rtree 
# Install Geopandas
# !pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
# !pip install descartes 

In [129]:
import pandas as pd 
import numpy as np
import geopandas as gpd 
from datetime import datetime

# Set maximum number of rows and columns that can be viewed
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 100)


# <a id='2'>Importing Shapefile</a>

<a href='#0'>Return to Top</a>

In [12]:
# import shapefile into dataframe
# ny_shp = gpd.read_file("/content/drive/MyDrive/Datasets/DataKind/NY/ny_spdf/ny_spdf.shp")
ny_shp = gpd.read_file("data/NY/ny_spdf/ny_spdf.shp")
ny_shp.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,36,071,000502,1400000US36071000502,36071000502,5.02,CT,967431.0,969216.0,"POLYGON ((-74.02226 41.49281, -74.02180 41.496..."
1,36,103,135208,1400000US36103135208,36103135208,1352.08,CT,2287077.0,0.0,"POLYGON ((-73.28263 40.83063, -73.28157 40.832..."
2,36,047,057800,1400000US36047057800,36047057800,578,CT,172233.0,0.0,"POLYGON ((-73.95398 40.60140, -73.95304 40.601..."
3,36,047,058900,1400000US36047058900,36047058900,589,CT,424025.0,38353.0,"POLYGON ((-73.94605 40.72926, -73.94419 40.729..."
4,36,055,013204,1400000US36055013204,36055013204,132.04,CT,28207247.0,93541.0,"POLYGON ((-77.66835 43.02829, -77.66806 43.029..."


In [13]:
# get some information about GEOID
print("Number of unique values: ", ny_shp.GEOID.nunique())
print("Minimum value: ", ny_shp.GEOID.min())
print("Maximum value: ", ny_shp.GEOID.max())

Number of unique values:  4906
Minimum value:  36001000100
Maximum value:  36123150500


In [16]:
# get some information about TRACTCE
print("Number of unique values: ", ny_shp.TRACTCE.nunique())
print("Minimum value: ", ny_shp.TRACTCE.min())
print("Maximum value: ", ny_shp.TRACTCE.max())

Number of unique values:  2697
Minimum value:  000100
Maximum value:  990100


In [15]:
# sort dataframe by GEOID values
ny_shape = ny_shp.sort_values(by=['GEOID'])
ny_shape.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
539,36,001,000100,1400000US36001000100,36001000100,1,CT,2367456.0,245021.0,"POLYGON ((-73.74506 42.67228, -73.74374 42.677..."
2748,36,001,000200,1400000US36001000200,36001000200,2,CT,2065161.0,0.0,"POLYGON ((-73.75993 42.65997, -73.76156 42.660..."
3007,36,001,000300,1400000US36001000300,36001000300,3,CT,5758852.0,55326.0,"POLYGON ((-73.83021 42.69418, -73.82785 42.696..."
3515,36,001,000401,1400000US36001000401,36001000401,4.01,CT,9065815.0,86327.0,"POLYGON ((-73.89421 42.70977, -73.89668 42.711..."
2751,36,001,000403,1400000US36001000403,36001000403,4.03,CT,3138700.0,0.0,"POLYGON ((-73.82108 42.67857, -73.81916 42.680..."


In [58]:
ny_shape.COUNTYFP.nunique()

62

## Synopsis of NY Shapefile:

There are 4906 rows in the shapefile, as well as 4906 unique rows of GEOID's. The state FIPS code is 36, and there are 62 county FIPS code in NY.  The AFFGEOID seems to be the GEOID affixed with 1400000US in front of each GEOID.  There are values given for land area, water area, and most importantly the geometry of the tract boundaries. An LSAD of CT tells us that we are dealing on the level of the Census Tract.

Each GEOID consists of the state FIPS code, its county FIPS code, and then the tract FIPS code, which apparently in itself is not unique and can be repeated, so 2 + 3 + 6 digits in total for the three components or an 11-digit tract FIPS code that is unique for every tract.

The NAME represents a census tract code that split into a 4-digit and 2-digit suffix placed after the decimal. Census tract numbers can consist of up to 6 digits:  up to a 4-digit basic number and optional 2-digit suffix, i.e. 1457.02. When used as name, any leading zeroes are eliminated and the suffix is appended if designated. 

Within the standard census geographic hierarchy, census tracts never cross state or county boundaries, but may cross the boundaries of county subdivisions,
places, urban areas, voting districts, congressional districts. 

# <a id='3'>ACS Database</a>

<a href='#0'>Return to Top</a>

Pertinent Columns for analysis:

- `f_black` = fraction of population black
- `f_hispanic` = fraction of population Hispanic
- `f_ba` = fraction of population with Bachelor's
- `f_broadband` =  fraction of households with broadband access
- `f_computer` = fraction of households with computer access
- `population` = total population
- `mhi` = median household income



## Importing Database

In [14]:
# import database
# ny_acs = pd.read_csv('/content/drive/MyDrive/Datasets/DataKind/NY/acs_2019_NY.csv')
ny_acs = pd.read_csv('data/NY/acs_2019_NY.csv')
ny_acs.head(20)

,state,county,tract,geoid,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,households,broadband,computer,black,hispanic,mhi.1,ba,den_computers,n_computer,n_broadband,den_black,n_black,den_hispanic,n_hispanic,den_ba,n_ba,nhh_computer,nhh_broadband,nhh_computer_any_internet,nhh_computer_and_dialup,nhh_computer_and_broadband,nhh_computer_no_internet,nhh_no_computer,den_age,n_children,n_children_computer,n_children_computer_and_dialup,n_children_computer_and_broadband,n_children_computer_no_internet,n_children_no_computer,state_lkp
0,36,1,100,36001000100,0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0,800.0,83.6,89.6,74.0,12.8,32389.0,13.7,800.0,717.0,669.0,2035.0,1506.0,2035.0,261.0,1283.0,176.0,717,669,717,0,669,48,83,2022,532,520,0,511,9,12,36
1,36,1,200,36001000200,0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0,2213.0,76.7,90.3,84.5,4.3,27714.0,22.5,2213.0,1999.0,1697.0,4793.0,4050.0,4793.0,206.0,2710.0,609.0,1999,1697,1999,0,1685,314,214,4711,1274,1221,0,1138,83,53,36
2,36,1,300,36001000300,0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0,2362.0,81.9,88.1,43.5,12.1,45272.0,28.9,2362.0,2081.0,1935.0,6147.0,2674.0,6147.0,742.0,3725.0,1078.0,2081,1935,2081,11,1917,153,281,5925,1729,1717,0,1630,87,12,36
3,36,1,401,36001000401,0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0,997.0,83.7,85.7,7.6,4.6,74274.0,47.6,997.0,854.0,834.0,2362.0,180.0,2362.0,109.0,2169.0,1032.0,854,834,854,0,826,28,143,1790,171,171,0,171,0,0,36
4,36,1,403,36001000403,0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0,2186.0,81.7,86.1,12.9,5.8,74426.0,59.1,2186.0,1882.0,1786.0,4253.0,548.0,4253.0,246.0,3220.0,1904.0,1882,1786,1882,0,1733,149,304,4240,462,462,0,462,0,0,36
5,36,1,404,36001000404,1.000,1.000,0.073,0.225,0.190,NaN,-666666666.0,5082.0,15.0,100.0,100.0,22.5,19.0,-666666666.0,7.3,15.0,15.0,15.0,5082.0,1143.0,5082.0,965.0,55.0,4.0,15,15,15,0,15,0,0,15,0,0,0,0,0,0,36
6,36,1,501,36001000501,0.718,0.811,0.228,0.439,0.226,10.38,32165.0,3523.0,1433.0,71.8,81.1,43.9,22.6,32165.0,22.8,1433.0,1162.0,1029.0,3523.0,1547.0,3523.0,795.0,2192.0,499.0,1162,1029,1162,0,1018,144,271,3440,917,881,0,721,160,36,36
7,36,1,502,36001000502,0.806,0.945,0.621,0.201,0.106,11.01,60263.0,3785.0,942.0,80.6,94.5,20.1,10.6,60263.0,62.1,942.0,890.0,759.0,3785.0,761.0,3785.0,402.0,1171.0,727.0,890,759,890,0,759,131,52,1892,212,212,0,183,29,0,36
8,36,1,600,36001000600,0.747,0.912,0.183,0.363,0.137,10.01,22298.0,3663.0,1408.0,74.7,91.2,36.3,13.7,22298.0,18.3,1408.0,1284.0,1052.0,3663.0,1328.0,3663.0,502.0,1742.0,319.0,1284,1052,1284,20,1052,212,124,3565,865,705,0,659,46,160,36
9,36,1,700,36001000700,0.751,0.834,0.167,0.807,0.099,10.51,36639.0,3885.0,1485.0,75.1,83.4,80.7,9.9,36639.0,16.7,1485.0,1238.0,1115.0,3885.0,3136.0,3885.0,386.0,2442.0,409.0,1238,1115,1238,18,1087,133,247,3868,1054,984,0,921,63,70,36


## Data Cleaning

In [17]:
ny_acs.columns

Index(['state', 'county', 'tract', 'geoid', 'f_broadband', 'f_computer',
       'f_ba', 'f_black', 'f_hispanic', 'log_mhi', 'mhi', 'population',
       'households', 'broadband', 'computer', 'black', 'hispanic', 'mhi.1',
       'ba', 'den_computers', 'n_computer', 'n_broadband', 'den_black',
       'n_black', 'den_hispanic', 'n_hispanic', 'den_ba', 'n_ba',
       'nhh_computer', 'nhh_broadband', 'nhh_computer_any_internet',
       'nhh_computer_and_dialup', 'nhh_computer_and_broadband',
       'nhh_computer_no_internet', 'nhh_no_computer', 'den_age', 'n_children',
       'n_children_computer', 'n_children_computer_and_dialup',
       'n_children_computer_and_broadband', 'n_children_computer_no_internet',
       'n_children_no_computer', 'state_lkp'],
      dtype='object')

In [18]:
ny_acs.drop(columns=['state', 'county', 'tract', 'broadband', 'computer', 'black', 'hispanic', 'mhi.1', 'den_computers', 
       'n_black', 'n_hispanic', 'den_black', 'den_hispanic', 'state_lkp', 'den_age', 'n_computer', 'n_broadband', 
       'state_lkp', 'nhh_computer_any_internet', 
       'nhh_computer_and_dialup', 'nhh_computer_and_broadband',
       'nhh_computer_no_internet', 'nhh_no_computer', 'n_children',
       'n_children_computer', 'n_children_computer_and_dialup',
       'n_children_computer_and_broadband', 'n_children_computer_no_internet',
       'n_children_no_computer', 'households', 'ba', 'den_ba', 'n_ba', 'nhh_computer', 'nhh_broadband'], inplace=True)
ny_acs.head()

,geoid,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population
0,36001000100,0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0
1,36001000200,0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0
2,36001000300,0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0
3,36001000401,0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0
4,36001000403,0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0


In [19]:
ny_acs[ny_acs < 0] = np.nan

In [20]:
ny_acs.isna().sum()

geoid            0
f_broadband     90
f_computer      90
f_ba            68
f_black         64
f_hispanic      64
log_mhi        133
mhi            133
population       0
dtype: int64

In [21]:
# get some information about geoid
print("Number of unique values: ", ny_acs.geoid.nunique())
print("Minimum value: ", ny_acs.geoid.min())
print("Maximum value: ", ny_acs.geoid.max())


Number of unique values:  4918
Minimum value:  36001000100
Maximum value:  36123150500


In [23]:
ny_acs['geoid'] = ny_acs['geoid'].apply(lambda x: str(x))

In [24]:
set(ny_acs.geoid.values).symmetric_difference(set(ny_shape.GEOID.values))

{'36011990200',
 '36013990000',
 '36055990000',
 '36059990100',
 '36059990200',
 '36059990301',
 '36059990302',
 '36059990400',
 '36063990000',
 '36073990000',
 '36075990000',
 '36085990100'}

There are more Geoids in the ACS dataset than in the shapefile.  4918 vs. 4906.

## Joining Shapefile with ACS Database

In [25]:
joined_df = ny_shape.merge(ny_acs, how='left', left_on='GEOID', right_on='geoid')
joined_df.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,geoid,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population
0,36,001,000100,1400000US36001000100,36001000100,1,CT,2367456.0,245021.0,"POLYGON ((-73.74506 42.67228, -73.74374 42.677...",36001000100,0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0
1,36,001,000200,1400000US36001000200,36001000200,2,CT,2065161.0,0.0,"POLYGON ((-73.75993 42.65997, -73.76156 42.660...",36001000200,0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0
2,36,001,000300,1400000US36001000300,36001000300,3,CT,5758852.0,55326.0,"POLYGON ((-73.83021 42.69418, -73.82785 42.696...",36001000300,0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0
3,36,001,000401,1400000US36001000401,36001000401,4.01,CT,9065815.0,86327.0,"POLYGON ((-73.89421 42.70977, -73.89668 42.711...",36001000401,0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0
4,36,001,000403,1400000US36001000403,36001000403,4.03,CT,3138700.0,0.0,"POLYGON ((-73.82108 42.67857, -73.81916 42.680...",36001000403,0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0


In [26]:
joined_df.drop(columns=['STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'NAME', 'LSAD', 'ALAND', 'AWATER', 'geoid'], inplace=True)
joined_df.head()

,GEOID,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population
0,36001000100,"POLYGON ((-73.74506 42.67228, -73.74374 42.677...",0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0
1,36001000200,"POLYGON ((-73.75993 42.65997, -73.76156 42.660...",0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0
2,36001000300,"POLYGON ((-73.83021 42.69418, -73.82785 42.696...",0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0
3,36001000401,"POLYGON ((-73.89421 42.70977, -73.89668 42.711...",0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0
4,36001000403,"POLYGON ((-73.82108 42.67857, -73.81916 42.680...",0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0


In [27]:
joined_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4906 entries, 0 to 4905
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   GEOID        4906 non-null   object  
 1   geometry     4906 non-null   geometry
 2   f_broadband  4828 non-null   float64 
 3   f_computer   4828 non-null   float64 
 4   f_ba         4850 non-null   float64 
 5   f_black      4854 non-null   float64 
 6   f_hispanic   4854 non-null   float64 
 7   log_mhi      4785 non-null   float64 
 8   mhi          4785 non-null   float64 
 9   population   4906 non-null   float64 
dtypes: float64(8), geometry(1), object(1)
memory usage: 421.6+ KB


There are currrently 4906 instances corresponding each of the census tracts in the shapefile.

# <a id='4'>FCC Database</a>

<a href='#0'>Return to Top</a>

The columns are as follows:

- `tract`: 11-digit tract FIPS code
- `max_dn`: average maximum advertised downstream speed offered by provider in census tract by block
- `max_up`: average maximum advertised upstream speed offered by provider in census tract by block
- `dn10`:	average count of providers offering advertised downstream greater than 10 by block
- `dn100`: average count of providers offering advertised downstream greater than 100 by block
- `dn250`: average count of providers offering advertised downstream greater than 250 by block
- `fiber_100u`: average count of providers offiering fiber by block



## Importing Database

In [28]:
# importing FCC database
# ny_fcc = pd.read_csv('/content/drive/MyDrive/Datasets/DataKind/NY/fcc_477_census_tract_NY.csv')
ny_fcc = pd.read_csv('data/NY/fcc_477_census_tract_NY.csv')

ny_fcc.head()

,tract,max_dn,max_up,dn10,dn100,dn250,fiber_100u,state
0,36001000100,711.250000,50.722222,1.875000,0.777778,0.777778,0.027778,36
1,36001000200,940.000000,35.000000,2.529412,1.011765,1.000000,0.000000,36
2,36001000300,928.040541,98.195946,2.290541,1.060811,1.060811,0.074324,36
3,36001000401,928.975904,65.168675,2.156627,1.024096,1.024096,0.036145,36
4,36001000403,940.000000,43.203883,2.310680,1.009709,1.009709,0.009709,36


## Data Cleaning

In [29]:
# check for any null values in dataframe
ny_fcc.isna().sum()

tract         0
max_dn        0
max_up        0
dn10          0
dn100         0
dn250         0
fiber_100u    0
state         0
dtype: int64

In [30]:
# show value counts of tract column
len(ny_fcc.tract.value_counts())

4900

In [31]:
# change tract column to string type for joining
ny_fcc['tract'] = ny_fcc['tract'].apply(lambda x: str(x))
type(ny_fcc['tract'][0])

str

## Merging FCC Database with ACS and Shapefile

In [32]:
# merge two database on geoid
joined_2 = joined_df.merge(ny_fcc, how='left', left_on='GEOID', right_on='tract')
joined_2.head()

,GEOID,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,tract,max_dn,max_up,dn10,dn100,dn250,fiber_100u,state
0,36001000100,"POLYGON ((-73.74506 42.67228, -73.74374 42.677...",0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0,36001000100,711.250000,50.722222,1.875000,0.777778,0.777778,0.027778,36.0
1,36001000200,"POLYGON ((-73.75993 42.65997, -73.76156 42.660...",0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0,36001000200,940.000000,35.000000,2.529412,1.011765,1.000000,0.000000,36.0
2,36001000300,"POLYGON ((-73.83021 42.69418, -73.82785 42.696...",0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0,36001000300,928.040541,98.195946,2.290541,1.060811,1.060811,0.074324,36.0
3,36001000401,"POLYGON ((-73.89421 42.70977, -73.89668 42.711...",0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0,36001000401,928.975904,65.168675,2.156627,1.024096,1.024096,0.036145,36.0
4,36001000403,"POLYGON ((-73.82108 42.67857, -73.81916 42.680...",0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0,36001000403,940.000000,43.203883,2.310680,1.009709,1.009709,0.009709,36.0


In [33]:
joined_2.drop(columns=['tract', 'state', 'dn10', 'dn100', 'dn250', 'fiber_100u'], inplace=True)
joined_2.head()

,GEOID,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,max_dn,max_up
0,36001000100,"POLYGON ((-73.74506 42.67228, -73.74374 42.677...",0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0,711.250000,50.722222
1,36001000200,"POLYGON ((-73.75993 42.65997, -73.76156 42.660...",0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0,940.000000,35.000000
2,36001000300,"POLYGON ((-73.83021 42.69418, -73.82785 42.696...",0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0,928.040541,98.195946
3,36001000401,"POLYGON ((-73.89421 42.70977, -73.89668 42.711...",0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0,928.975904,65.168675
4,36001000403,"POLYGON ((-73.82108 42.67857, -73.81916 42.680...",0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0,940.000000,43.203883


## Synopsis of FCC Database

FCC has 4900 rows of data, and the shapefile had 4906 rows of each of the tracts, and every row has a unique tract value. Here `tract` is synonymous with `GEOID`. It would seem we are missing data for a few GEOIDs, but that remains to be seen how our analysis will be affected.

# <a id='5'>Ookla Database</a>

<a href='#0'>Return to Top</a>

The column keys are:
- `avg_d_kbps`, `int`, average download speed of all tests performed in the tile, represented in kilobits per second.
- `avg_u_kbps`, `int`, 	average upload speed of all tests performed in the tile, represented in kilobits per second.
- `avg_lat_ms`, `int`, average latency of all tests performed in the tile, represented in milliseconds
- `tests`, `int`, number of tests taken in the tile.
- `devices`, `int`, number of unique devices contributing tests in the tile.
- `quadkey`: quadkey representing the tile.



## Downloading Ookla Mobile and Fixed Datasets

The provided database was not a geodataframe, and in order to perform a spatial join, I need two geodataframes, so I used the following code to download the Ookla files.  The resulting downloads were geodataframes, but had no geographic identifier in common with the other dataframes other than geometry and quadkey.

I performed a spatial join with the NY shapefile to filter the rows that are not within the boundaries of the state in question.

In [127]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_fixed_tiles.zip"
    return url

In [131]:
fixed_url = get_tile_url("fixed", 2019, 2)
fixed_ookla_shp = gpd.read_file(fixed_url)
fixed_ookla_shp.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,3112231122203110,37621,18081,13,105,35,"POLYGON ((144.92065 -37.61423, 144.92615 -37.6..."
1,0231123033301330,165881,30946,10,71,5,"POLYGON ((-97.13013 33.25706, -97.12463 33.257..."
2,0231033130220100,158905,142679,14,8,6,"POLYGON ((-101.93115 33.50476, -101.92566 33.5..."
3,0233102110002012,25166,5245,54,351,42,"POLYGON ((-99.12964 19.26448, -99.12415 19.264..."
4,0231322223022233,19321,6858,47,110,21,"POLYGON ((-100.88196 22.11109, -100.87646 22.1..."


In [35]:
fixed_ookla_shp.shape

(4857535, 7)

In [36]:
mobile_url = get_tile_url("mobile", 2019, 2)
mobile_ookla_shp = gpd.read_file(mobile_url)
mobile_ookla_shp.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,1231320120100310,9132,8713,54,18,12,"POLYGON ((80.36499 26.41155, 80.37048 26.41155..."
1,1321001210112203,48612,9973,36,3,3,"POLYGON ((116.28479 39.85494, 116.29028 39.854..."
2,1230220321222323,32744,6136,22,13,8,"POLYGON ((46.78528 24.85155, 46.79077 24.85155..."
3,0320120321303112,34523,8544,102,4,3,"POLYGON ((-76.73950 34.70098, -76.73401 34.700..."
4,1220110031123022,9767,6629,68,2,1,"POLYGON ((18.14941 40.33398, 18.15491 40.33398..."


In [37]:
mobile_ookla_shp.shape

(3340189, 7)

# <a id='6'>Spatial Join</a>

<a href='#0'>Return to Top</a>

**predicate or ops:**
- *intersects*
    - Returns True if the boundary or interior of the object intersect in any way with those of the other. In other words, geometric objects intersect if they have any boundary or interior point in common.
- *contains*
    - Returns True if no points of other lie in the exterior of the object and at least one point of the interior of other lies in the interior of object. This predicate applies to all types, and is inverse to within(). The expression a.contains(b) == b.within(a) always evaluates to True.
- *within*
    - Returns True if the object’s boundary and interior intersect only with the interior of the other (not its boundary or exterior). This applies to all types and is the inverse of contains().
- *touches*
    - Returns True if the objects have at least one point in common and their interiors do not intersect with any part of the other. Overlapping features do not therefore touch, another potential “gotcha”. For example, the following lines touch at (1, 1), but do not overlap.
- *crosses*
    - Returns True if the interior of the object intersects the interior of the other but does not contain it, and the dimension of the intersection is less than the dimension of the one or the other.
- *overlaps*
    - Returns True if the geometries have more than one but not all points in common, have the same dimension, and the intersection of the interiors of the geometries has the same dimension as the geometries themselves

**how:**
- left: use the index from the first (or left_df) GeoDataFrame that you provide to GeoDataFrame.sjoin(); retain only the left_df geometry column

- right: use index from second (or right_df); retain only the right_df geometry column

- inner: use intersection of index values from both GeoDataFrame; retain only the left_df geometry column

## Importing Census Shapefile

In [39]:
# us_tracts = gpd.read_file('data/z-archive/Census Tract Shapefiles/cb_2019_us_tract_500k/cb_2019_us_tract_500k.shp')
us_tracts = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/DataKind/cb_2019_us_tract_500k/cb_2019_us_tract_500k.shp')

ny_tracts = us_tracts[us_tracts['STATEFP']=='36']

ny_tracts = ny_tracts.to_crs("EPSG:4326")

ny_tracts.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
2,36,071,000502,1400000US36071000502,36071000502,5.02,CT,967431,969216,"POLYGON ((-74.02226 41.49281, -74.02180 41.496..."
10,36,103,135208,1400000US36103135208,36103135208,1352.08,CT,2287077,0,"POLYGON ((-73.28263 40.83063, -73.28157 40.832..."
49,36,047,057800,1400000US36047057800,36047057800,578,CT,172233,0,"POLYGON ((-73.95398 40.60140, -73.95304 40.601..."
50,36,047,058900,1400000US36047058900,36047058900,589,CT,424025,38353,"POLYGON ((-73.94605 40.72926, -73.94419 40.729..."
53,36,055,013204,1400000US36055013204,36055013204,132.04,CT,28207247,93541,"POLYGON ((-77.66835 43.02829, -77.66806 43.029..."


## Fixed Broadband

In [41]:
fixed_bb_ny = gpd.sjoin(fixed_ookla_shp, ny_tracts, how='inner', op='intersects')
fixed_bb_ny = fixed_bb_ny.reset_index(drop=True)
fixed_bb_ny.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,0320101101323112,136203,40848,11,248,97,"POLYGON ((-73.92700 40.74310, -73.92151 40.743...",1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
1,0320101101323130,153640,63816,10,268,70,"POLYGON ((-73.92700 40.73893, -73.92151 40.738...",1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
2,0320101101323113,156486,51539,12,266,82,"POLYGON ((-73.92151 40.74310, -73.91602 40.743...",1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
3,0320101101323131,158302,45390,11,274,75,"POLYGON ((-73.92151 40.73893, -73.91602 40.738...",1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
4,0320101101323112,136203,40848,11,248,97,"POLYGON ((-73.92700 40.74310, -73.92151 40.743...",59157,36,081,018900,1400000US36081018900,36081018900,189,CT,217707,0


In [42]:
fixed_bb_ny.GEOID.value_counts()

36103190704    241
36103200902    235
36059517701    173
36103190707    163
36045060200    133
              ... 
36037940100      1
36029940100      1
36061023900      1
36005017902      1
36027640001      1
Name: GEOID, Length: 4898, dtype: int64

In [43]:
fixed_bb_ny.shape

(89763, 17)

## Mobile Broadband

In [44]:
mobile_bb_ny = gpd.sjoin(mobile_ookla_shp, ny_tracts, how='inner', op='intersects')
mobile_bb_ny = mobile_bb_ny.reset_index(drop=True)
mobile_bb_ny.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,0320101101302301,44327,11950,34,61,22,"POLYGON ((-73.97644 40.79718, -73.97095 40.797...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
1,0320101101302303,67156,18470,35,61,25,"POLYGON ((-73.97644 40.79302, -73.97095 40.793...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
2,0320101101302312,42945,10367,27,54,22,"POLYGON ((-73.97095 40.79302, -73.96545 40.793...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
3,0320101101302310,72812,10699,45,30,13,"POLYGON ((-73.97095 40.79718, -73.96545 40.797...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
4,0320101101302313,53723,11901,109,5,3,"POLYGON ((-73.96545 40.79302, -73.95996 40.793...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0


In [45]:
mobile_bb_ny.shape

(32642, 17)

In [46]:
mobile_bb_ny.GEOID.value_counts()

36081071600    70
36059300900    50
36055013104    46
36103112206    44
36059407301    43
               ..
36103111803     1
36111950900     1
36027110004     1
36029016200     1
36025970900     1
Name: GEOID, Length: 4852, dtype: int64

Save results since running spatial join is somewhat time consuming

In [47]:
fixed_bb_ny.to_file('data/NY/fixed_bb_ny.geojson', driver='GeoJSON')
mobile_bb_ny.to_file('data/NY/mobile_bb_ny.geojson', driver='GeoJSON')

In [ ]:
fixed_bb_ny = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/DataKind/fixed_bb_ny.geojson')
mobile_bb_ny = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/DataKind/mobile_bb_ny.geojson')

## Aggregating Ookla Datasets

In [50]:
wm = lambda x: np.average(x, weights=mobile_bb_ny.loc[x.index, 'tests'])

mobile_agg = mobile_bb_ny.groupby(['GEOID'], as_index=False).agg(
                            m_d_kbps_wm = ('avg_d_kbps', wm),
                            m_u_kbps_wm = ('avg_u_kbps', wm),
                            m_tests = ('tests', sum),
                            m_devices = ('devices', sum)
                            )
mobile_agg.head()

,GEOID,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices
0,36001000100,33798.250000,9721.850000,20,15
1,36001000200,25220.151515,10305.969697,66,34
2,36001000300,46350.592105,11667.434211,76,51
3,36001000401,48040.646018,15980.106195,113,40
4,36001000403,51088.952381,11992.500000,42,28


In [51]:
wm = lambda x: np.average(x, weights=fixed_bb_ny.loc[x.index, 'tests'])

fixed_agg = fixed_bb_ny.groupby(['GEOID'], as_index=False).agg(
                            f_d_kbps_wm = ('avg_d_kbps', wm),
                            f_u_kbps_wm = ('avg_u_kbps', wm),
                            f_tests = ('tests', sum),
                            f_devices = ('devices', sum)
                            )
fixed_agg.head()

,GEOID,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,36001000100,53064.355263,13516.934211,76,29
1,36001000200,82315.136364,11795.281818,110,61
2,36001000300,61513.941704,9973.273543,223,87
3,36001000401,87434.738462,49736.338462,65,39
4,36001000403,86733.135135,13435.054054,185,86


In [52]:
df_ookla = mobile_agg.merge(fixed_agg, how='outer', on='GEOID')
df_ookla.head()

,GEOID,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,36001000100,33798.250000,9721.850000,20.0,15.0,53064.355263,13516.934211,76,29
1,36001000200,25220.151515,10305.969697,66.0,34.0,82315.136364,11795.281818,110,61
2,36001000300,46350.592105,11667.434211,76.0,51.0,61513.941704,9973.273543,223,87
3,36001000401,48040.646018,15980.106195,113.0,40.0,87434.738462,49736.338462,65,39
4,36001000403,51088.952381,11992.500000,42.0,28.0,86733.135135,13435.054054,185,86


In [53]:
df_ookla.shape

(4898, 9)

## Merging Ookla with other dataset

In [54]:
df_ookla['GEOID'] = df_ookla['GEOID'].apply(lambda x: str(x))
df_combined = joined_2.merge(df_ookla, how='left', on='GEOID')
df_combined.head()

,GEOID,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,max_dn,max_up,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,36001000100,"POLYGON ((-73.74506 42.67228, -73.74374 42.677...",0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0,711.250000,50.722222,33798.250000,9721.850000,20.0,15.0,53064.355263,13516.934211,76.0,29.0
1,36001000200,"POLYGON ((-73.75993 42.65997, -73.76156 42.660...",0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0,940.000000,35.000000,25220.151515,10305.969697,66.0,34.0,82315.136364,11795.281818,110.0,61.0
2,36001000300,"POLYGON ((-73.83021 42.69418, -73.82785 42.696...",0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0,928.040541,98.195946,46350.592105,11667.434211,76.0,51.0,61513.941704,9973.273543,223.0,87.0
3,36001000401,"POLYGON ((-73.89421 42.70977, -73.89668 42.711...",0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0,928.975904,65.168675,48040.646018,15980.106195,113.0,40.0,87434.738462,49736.338462,65.0,39.0
4,36001000403,"POLYGON ((-73.82108 42.67857, -73.81916 42.680...",0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0,940.000000,43.203883,51088.952381,11992.500000,42.0,28.0,86733.135135,13435.054054,185.0,86.0


## Saving Dataset

In [57]:
df_combined.to_file('data/NY/ny_final.geojson', driver='GeoJSON')

# <a id='7'>Using Centroids as Geometry for Spatial Join</a>

<a href='#0'>Return to Top</a>

The concern about performing a spatial join on the census tract geometries and the quadkey geometries was the potential repeating of quadkey tiles if they overlapped with two different census tracts.  By converting the quadkey geometries to their centroids, we eliminate any potential overlap, and each quadkey tile would be joined with only one census tract geometry.

In [73]:
us_tracts = gpd.read_file('data/z-archive/Census Tract Shapefiles/cb_2019_us_tract_500k/cb_2019_us_tract_500k.shp')
# us_tracts = gpd.read_file('/content/drive/MyDrive/Colab Notebooks/DataKind/cb_2019_us_tract_500k/cb_2019_us_tract_500k.shp')
ny_tracts = us_tracts[us_tracts['STATEFP']=='36']
ny_tracts = ny_tracts.to_crs("EPSG:4326")
ny_tracts.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
2,36,071,000502,1400000US36071000502,36071000502,5.02,CT,967431,969216,"POLYGON ((-74.02226 41.49281, -74.02180 41.496..."
10,36,103,135208,1400000US36103135208,36103135208,1352.08,CT,2287077,0,"POLYGON ((-73.28263 40.83063, -73.28157 40.832..."
49,36,047,057800,1400000US36047057800,36047057800,578,CT,172233,0,"POLYGON ((-73.95398 40.60140, -73.95304 40.601..."
50,36,047,058900,1400000US36047058900,36047058900,589,CT,424025,38353,"POLYGON ((-73.94605 40.72926, -73.94419 40.729..."
53,36,055,013204,1400000US36055013204,36055013204,132.04,CT,28207247,93541,"POLYGON ((-77.66835 43.02829, -77.66806 43.029..."


In [77]:
fixed_ookla_shp.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,3112231122203110,37621,18081,13,105,35,"POLYGON ((144.92065 -37.61423, 144.92615 -37.6..."
1,0231123033301330,165881,30946,10,71,5,"POLYGON ((-97.13013 33.25706, -97.12463 33.257..."
2,0231033130220100,158905,142679,14,8,6,"POLYGON ((-101.93115 33.50476, -101.92566 33.5..."
3,0233102110002012,25166,5245,54,351,42,"POLYGON ((-99.12964 19.26448, -99.12415 19.264..."
4,0231322223022233,19321,6858,47,110,21,"POLYGON ((-100.88196 22.11109, -100.87646 22.1..."


In [78]:
fixed_ookla_shp = fixed_ookla_shp.to_crs("EPSG:4326")
fixed_ookla_shp['centroid'] = fixed_ookla_shp.centroid

/var/folders/0f/dj_hz1316w7c4qhdk33w3p3w0000gn/T/ipykernel_90012/1863881433.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fixed_ookla_shp['centroid'] = fixed_ookla_shp.centroid


In [79]:
mobile_ookla_shp = mobile_ookla_shp.to_crs("EPSG:4326")
mobile_ookla_shp['centroid'] = mobile_ookla_shp.centroid

/var/folders/0f/dj_hz1316w7c4qhdk33w3p3w0000gn/T/ipykernel_90012/1271778484.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mobile_ookla_shp['centroid'] = mobile_ookla_shp.centroid


In [90]:
fixed_ookla_shp.set_geometry('centroid')
fixed_bb_ny_2 = gpd.sjoin(fixed_ookla_shp, ny_tracts, op='intersects', how='inner')
fixed_bb_ny_2.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,centroid,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
13,0320101101323112,136203,40848,11,248,97,"POLYGON ((-73.92700 40.74310, -73.92151 40.743...",POINT (-73.92426 40.74101),1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
11349,0320101101323130,153640,63816,10,268,70,"POLYGON ((-73.92700 40.73893, -73.92151 40.738...",POINT (-73.92426 40.73685),1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
67666,0320101101323113,156486,51539,12,266,82,"POLYGON ((-73.92151 40.74310, -73.91602 40.743...",POINT (-73.91876 40.74101),1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
231739,0320101101323131,158302,45390,11,274,75,"POLYGON ((-73.92151 40.73893, -73.91602 40.738...",POINT (-73.91876 40.73685),1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
13,0320101101323112,136203,40848,11,248,97,"POLYGON ((-73.92700 40.74310, -73.92151 40.743...",POINT (-73.92426 40.74101),59157,36,081,018900,1400000US36081018900,36081018900,189,CT,217707,0


In [91]:
fixed_bb_ny_2.GEOID.nunique()

4898

In [92]:
fixed_bb_ny_2.shape

(89763, 18)

In [108]:
fixed_bb_ny_2.drop(columns=['centroid'], inplace=True)
fixed_bb_ny_2 = fixed_bb_ny_2.reset_index(drop=True)
fixed_bb_ny_2.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,0320101101323112,136203,40848,11,248,97,"POLYGON ((-73.92700 40.74310, -73.92151 40.743...",1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
1,0320101101323130,153640,63816,10,268,70,"POLYGON ((-73.92700 40.73893, -73.92151 40.738...",1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
2,0320101101323113,156486,51539,12,266,82,"POLYGON ((-73.92151 40.74310, -73.91602 40.743...",1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
3,0320101101323131,158302,45390,11,274,75,"POLYGON ((-73.92151 40.73893, -73.91602 40.738...",1075,36,081,018501,1400000US36081018501,36081018501,185.01,CT,98522,0
4,0320101101323112,136203,40848,11,248,97,"POLYGON ((-73.92700 40.74310, -73.92151 40.743...",59157,36,081,018900,1400000US36081018900,36081018900,189,CT,217707,0


In [109]:
fixed_bb_ny_2.to_file('data/NY/fixed_bb_ny_2.geojson', driver='GeoJSON')

In [99]:
mobile_ookla_shp.set_geometry('centroid')
mobile_bb_ny_2 = gpd.sjoin(mobile_ookla_shp, ny_tracts, how='inner', op='intersects')
mobile_bb_ny_2.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,centroid,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
577,0320101101302301,44327,11950,34,61,22,"POLYGON ((-73.97644 40.79718, -73.97095 40.797...",POINT (-73.97369 40.79510),7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
137385,0320101101302303,67156,18470,35,61,25,"POLYGON ((-73.97644 40.79302, -73.97095 40.793...",POINT (-73.97369 40.79094),7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
145171,0320101101302312,42945,10367,27,54,22,"POLYGON ((-73.97095 40.79302, -73.96545 40.793...",POINT (-73.96820 40.79094),7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
536383,0320101101302310,72812,10699,45,30,13,"POLYGON ((-73.97095 40.79718, -73.96545 40.797...",POINT (-73.96820 40.79510),7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
1339262,0320101101302313,53723,11901,109,5,3,"POLYGON ((-73.96545 40.79302, -73.95996 40.793...",POINT (-73.96271 40.79094),7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0


In [100]:
mobile_bb_ny_2.GEOID.nunique()

4852

In [101]:
mobile_bb_ny_2.shape

(32642, 18)

The sizes are the same with both methods of joining with the same number of GEOID's.

In [106]:
mobile_bb_ny_2.drop(columns=['centroid'], inplace=True)
mobile_bb_ny_2 = mobile_bb_ny_2.reset_index(drop=True)
mobile_bb_ny_2.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,0320101101302301,44327,11950,34,61,22,"POLYGON ((-73.97644 40.79718, -73.97095 40.797...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
1,0320101101302303,67156,18470,35,61,25,"POLYGON ((-73.97644 40.79302, -73.97095 40.793...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
2,0320101101302312,42945,10367,27,54,22,"POLYGON ((-73.97095 40.79302, -73.96545 40.793...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
3,0320101101302310,72812,10699,45,30,13,"POLYGON ((-73.97095 40.79718, -73.96545 40.797...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0
4,0320101101302313,53723,11901,109,5,3,"POLYGON ((-73.96545 40.79302, -73.95996 40.793...",7372,36,061,018100,1400000US36061018100,36061018100,181,CT,138438,0


In [107]:
mobile_bb_ny_2.to_file('data/NY/mobile_bb_ny_2.geojson', driver='GeoJSON')

In [117]:
mobile_bb_ny_2 = gpd.read_file('data/NY/mobile_bb_ny_2.geojson')
fixed_bb_ny_2 = gpd.read_file('data/NY/fixed_bb_ny_2.geojson')

## Aggregating Ookla Results for Joining with Main Dataset

In [118]:
wm = lambda x: np.average(x, weights=mobile_bb_ny_2.loc[x.index, 'tests'])

mobile_agg_2 = mobile_bb_ny_2.groupby(['GEOID'], as_index=False).agg(
                                    m_d_kbps_wm = ('avg_d_kbps', wm),
                                    m_u_kbps_wm = ('avg_u_kbps', wm),
                                    m_tests = ('tests', sum),
                                    m_devices = ('devices', sum)
                                    )
mobile_agg_2.head()

,GEOID,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices
0,36001000100,33798.250000,9721.850000,20,15
1,36001000200,25220.151515,10305.969697,66,34
2,36001000300,46350.592105,11667.434211,76,51
3,36001000401,48040.646018,15980.106195,113,40
4,36001000403,51088.952381,11992.500000,42,28


In [119]:
mobile_agg_2.GEOID.value_counts()

36001000100    1
36081019400    1
36081019000    1
36081018900    1
36081018800    1
              ..
36047067600    1
36047067400    1
36047067200    1
36047067000    1
36123150500    1
Name: GEOID, Length: 4852, dtype: int64

In [120]:
wm = lambda x: np.average(x, weights=fixed_bb_ny_2.loc[x.index, 'tests'])

fixed_agg_2 = fixed_bb_ny_2.groupby(['GEOID'], as_index=False).agg(
                                  f_d_kbps_wm = ('avg_d_kbps', wm),
                                  f_u_kbps_wm = ('avg_u_kbps', wm),
                                  f_tests = ('tests', sum),
                                  f_devices = ('devices', sum)
                                  )
fixed_agg_2.head()

,GEOID,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,36001000100,53064.355263,13516.934211,76,29
1,36001000200,82315.136364,11795.281818,110,61
2,36001000300,61513.941704,9973.273543,223,87
3,36001000401,87434.738462,49736.338462,65,39
4,36001000403,86733.135135,13435.054054,185,86


In [121]:
fixed_agg_2.GEOID.value_counts()

36001000100    1
36081018600    1
36081019600    1
36081019400    1
36081019200    1
              ..
36047067000    1
36047066600    1
36047066200    1
36047066000    1
36123150500    1
Name: GEOID, Length: 4898, dtype: int64

In [122]:
df_ookla_2 = mobile_agg_2.merge(fixed_agg_2, how='outer', on='GEOID')
df_ookla_2.head()

,GEOID,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,36001000100,33798.250000,9721.850000,20.0,15.0,53064.355263,13516.934211,76,29
1,36001000200,25220.151515,10305.969697,66.0,34.0,82315.136364,11795.281818,110,61
2,36001000300,46350.592105,11667.434211,76.0,51.0,61513.941704,9973.273543,223,87
3,36001000401,48040.646018,15980.106195,113.0,40.0,87434.738462,49736.338462,65,39
4,36001000403,51088.952381,11992.500000,42.0,28.0,86733.135135,13435.054054,185,86


## Merge Ookla Dataset with Main Dataset

In [123]:
df_ookla_2['GEOID'] = df_ookla_2['GEOID'].apply(lambda x: str(x))
df_combined_2 = joined_2.merge(df_ookla_2, how='left', on='GEOID')
df_combined_2.head()

,GEOID,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,max_dn,max_up,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,36001000100,"POLYGON ((-73.74506 42.67228, -73.74374 42.677...",0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0,711.250000,50.722222,33798.250000,9721.850000,20.0,15.0,53064.355263,13516.934211,76.0,29.0
1,36001000200,"POLYGON ((-73.75993 42.65997, -73.76156 42.660...",0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0,940.000000,35.000000,25220.151515,10305.969697,66.0,34.0,82315.136364,11795.281818,110.0,61.0
2,36001000300,"POLYGON ((-73.83021 42.69418, -73.82785 42.696...",0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0,928.040541,98.195946,46350.592105,11667.434211,76.0,51.0,61513.941704,9973.273543,223.0,87.0
3,36001000401,"POLYGON ((-73.89421 42.70977, -73.89668 42.711...",0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0,928.975904,65.168675,48040.646018,15980.106195,113.0,40.0,87434.738462,49736.338462,65.0,39.0
4,36001000403,"POLYGON ((-73.82108 42.67857, -73.81916 42.680...",0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0,940.000000,43.203883,51088.952381,11992.500000,42.0,28.0,86733.135135,13435.054054,185.0,86.0


In [124]:
df_combined_2.to_file('data/NY/df_combined_2.geojson', driver='GeoJSON')


In [132]:
df_combined_2.head()

,GEOID,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,max_dn,max_up,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,36001000100,"POLYGON ((-73.74506 42.67228, -73.74374 42.677...",0.836,0.896,0.137,0.740,0.128,10.39,32389.0,2035.0,711.250000,50.722222,33798.250000,9721.850000,20.0,15.0,53064.355263,13516.934211,76.0,29.0
1,36001000200,"POLYGON ((-73.75993 42.65997, -73.76156 42.660...",0.767,0.903,0.225,0.845,0.043,10.23,27714.0,4793.0,940.000000,35.000000,25220.151515,10305.969697,66.0,34.0,82315.136364,11795.281818,110.0,61.0
2,36001000300,"POLYGON ((-73.83021 42.69418, -73.82785 42.696...",0.819,0.881,0.289,0.435,0.121,10.72,45272.0,6147.0,928.040541,98.195946,46350.592105,11667.434211,76.0,51.0,61513.941704,9973.273543,223.0,87.0
3,36001000401,"POLYGON ((-73.89421 42.70977, -73.89668 42.711...",0.837,0.857,0.476,0.076,0.046,11.22,74274.0,2362.0,928.975904,65.168675,48040.646018,15980.106195,113.0,40.0,87434.738462,49736.338462,65.0,39.0
4,36001000403,"POLYGON ((-73.82108 42.67857, -73.81916 42.680...",0.817,0.861,0.591,0.129,0.058,11.22,74426.0,4253.0,940.000000,43.203883,51088.952381,11992.500000,42.0,28.0,86733.135135,13435.054054,185.0,86.0


Need to update Illinois and Florida code still to reflect recent changes

# <a id='8'>Illinois Datasets</a>

<a href='#0'>Return to Top</a>

In [113]:
# shapefile = "/content/drive/MyDrive/Datasets/Broadband/IL/il_spdf/il_spdf.shp"
# acsfile = '/content/drive/MyDrive/Datasets/Broadband/IL/acs_2019_IL.csv'
# fccfile = '/content/drive/MyDrive/Datasets/Broadband/IL/fcc_477_census_tract_IL.csv'
# ooklafile = '/content/drive/MyDrive/Datasets/Broadband/IL/ookla_combined_il.csv'

shapefile = "data/IL/il_spdf/il_spdf.shp"
acsfile = 'data/IL/acs_2019_IL.csv'
fccfile = 'data/IL/fcc_477_census_tract_IL.csv'
FIPS='17'


In [114]:
shpfile = gpd.read_file(shapefile)
shpfile = shpfile.sort_values(by=['GEOID'])
shpfile.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
1992,17,001,000100,1400000US17001000100,17001000100,1,CT,12937185.0,22042.0,"POLYGON ((-91.37766 39.94160, -91.37759 39.946..."
1980,17,001,000201,1400000US17001000201,17001000201,2.01,CT,3206335.0,0.0,"POLYGON ((-91.39646 39.95621, -91.39631 39.965..."
22,17,001,000202,1400000US17001000202,17001000202,2.02,CT,1514956.0,0.0,"POLYGON ((-91.39370 39.94678, -91.37759 39.946..."
1117,17,001,000400,1400000US17001000400,17001000400,4,CT,4458010.0,211712.0,"POLYGON ((-91.42005 39.95081, -91.41917 39.951..."
784,17,001,000500,1400000US17001000500,17001000500,5,CT,1262517.0,0.0,"POLYGON ((-91.40340 39.95048, -91.39655 39.950..."


In [116]:
data = pd.read_csv(acsfile)
data = data.rename(columns={'geoid':'GEOID'})
data.drop(columns=['state', 'county', 'tract', 'broadband', 'computer', 'black', 'hispanic', 'mhi.1', 'den_computers', 
       'n_black', 'n_hispanic', 'den_black', 'den_hispanic', 'state_lkp', 'den_age', 'n_computer', 'n_broadband', 
       'state_lkp', 'nhh_computer_any_internet', 
       'nhh_computer_and_dialup', 'nhh_computer_and_broadband',
       'nhh_computer_no_internet', 'nhh_no_computer', 'n_children',
       'n_children_computer', 'n_children_computer_and_dialup',
       'n_children_computer_and_broadband', 'n_children_computer_no_internet',
       'n_children_no_computer'], inplace=True)
data[data < 0] = np.nan
data['GEOID'] = data['GEOID'].apply(lambda x: str(x))
data.head()

,GEOID,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,households,ba,den_ba,n_ba,nhh_computer,nhh_broadband
0,17001000100,0.813,0.861,0.246,0.068,0.000,10.78,48088.0,4761.0,2202.0,24.6,3381.0,831.0,1896,1791
1,17001000201,0.879,0.940,0.266,0.078,0.016,10.71,44855.0,1985.0,889.0,26.6,1400.0,373.0,836,781
2,17001000202,0.863,0.933,0.221,0.080,0.008,10.83,50375.0,2376.0,970.0,22.1,1527.0,338.0,905,837
3,17001000400,0.691,0.780,0.094,0.182,0.006,10.31,30164.0,3422.0,1316.0,9.4,2536.0,238.0,1027,909
4,17001000500,0.665,0.787,0.113,0.133,0.035,10.62,41008.0,2175.0,813.0,11.3,1455.0,165.0,640,541


In [117]:
data2 = pd.read_csv(fccfile)
data2 = data2.rename(columns={'tract':'GEOID'})
data2['GEOID'] = data2['GEOID'].apply(lambda x: str(x))
data2.drop(columns=['state'], inplace=True)
data2.head()

,GEOID,max_dn,max_up,dn10,dn100,dn250,fiber_100u
0,17001000100,931.548571,915.160000,2.268571,1.502857,1.502857,0.920000
1,17001000201,872.587302,796.936508,2.761905,1.603175,1.603175,0.809524
2,17001000202,1000.000000,1000.000000,3.944444,2.018519,2.018519,1.018519
3,17001000400,519.804124,198.982021,1.757732,0.680412,0.680412,0.190722
4,17001000500,982.120690,916.275862,3.775862,1.793103,1.793103,0.913793


In [118]:
il_tracts = us_tracts.loc[us_tracts['STATEFP']=='17'].to_crs(4326).reset_index(drop=True)
fixed_tiles_il = gpd.sjoin(fixed_ookla_shp, il_tracts, how='inner', op='intersects')
fixed_il = fixed_tiles_il.reset_index(drop=True)
mobile_tiles_il = gpd.sjoin(mobile_ookla_shp, il_tracts, how='inner', op='intersects')
mobile_il = mobile_tiles_il.reset_index(drop=True)

wm = lambda x: np.average(x, weights=mobile_il.loc[x.index, 'tests'])
mobile_agg = mobile_il.groupby(['GEOID'], as_index=False).agg(
                            m_d_kbps_wm = ('avg_d_kbps', wm),
                            m_u_kbps_wm = ('avg_u_kbps', wm),
                            m_tests = ('tests', sum),
                            m_devices = ('devices', sum)
                            )
wm = lambda x: np.average(x, weights=fixed_il.loc[x.index, 'tests'])
fixed_agg = fixed_il.groupby(['GEOID'], as_index=False).agg(
                            f_d_kbps_wm = ('avg_d_kbps', wm),
                            f_u_kbps_wm = ('avg_u_kbps', wm),
                            f_tests = ('tests', sum),
                            f_devices = ('devices', sum)
                            )
fixed_agg['GEOID'] = fixed_agg['GEOID'].apply(lambda x: int(x))
mobile_agg['GEOID'] = mobile_agg['GEOID'].apply(lambda x: int(x))
data3 = mobile_agg.merge(fixed_agg, how='outer', on='GEOID')
data3['GEOID'] = data3['GEOID'].apply(lambda x: str(x))

In [119]:
df_1 = shpfile.merge(data, how="left", on="GEOID")
df_2 = df_1.merge(data2, how="left", on="GEOID")
df_final = df_2.merge(data3, how="left", on="GEOID")
df_final.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,households,ba,den_ba,n_ba,nhh_computer,nhh_broadband,max_dn,max_up,dn10,dn100,dn250,fiber_100u,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,17,001,000100,1400000US17001000100,17001000100,1,CT,12937185.0,22042.0,"POLYGON ((-91.37766 39.94160, -91.37759 39.946...",0.813,0.861,0.246,0.068,0.000,10.78,48088.0,4761.0,2202.0,24.6,3381.0,831.0,1896,1791,931.548571,915.160000,2.268571,1.502857,1.502857,0.920000,45622.325000,10624.075000,40.0,30.0,83500.869110,52515.041885,191.0,92.0
1,17,001,000201,1400000US17001000201,17001000201,2.01,CT,3206335.0,0.0,"POLYGON ((-91.39646 39.95621, -91.39631 39.965...",0.879,0.940,0.266,0.078,0.016,10.71,44855.0,1985.0,889.0,26.6,1400.0,373.0,836,781,872.587302,796.936508,2.761905,1.603175,1.603175,0.809524,24662.416667,5445.083333,12.0,7.0,103008.682403,48455.686695,233.0,74.0
2,17,001,000202,1400000US17001000202,17001000202,2.02,CT,1514956.0,0.0,"POLYGON ((-91.39370 39.94678, -91.37759 39.946...",0.863,0.933,0.221,0.080,0.008,10.83,50375.0,2376.0,970.0,22.1,1527.0,338.0,905,837,1000.000000,1000.000000,3.944444,2.018519,2.018519,1.018519,31500.333333,1856.000000,3.0,3.0,99521.493976,60237.301205,166.0,62.0
3,17,001,000400,1400000US17001000400,17001000400,4,CT,4458010.0,211712.0,"POLYGON ((-91.42005 39.95081, -91.41917 39.951...",0.691,0.780,0.094,0.182,0.006,10.31,30164.0,3422.0,1316.0,9.4,2536.0,238.0,1027,909,519.804124,198.982021,1.757732,0.680412,0.680412,0.190722,17929.500000,4026.285714,28.0,17.0,86031.362416,25337.281879,149.0,62.0
4,17,001,000500,1400000US17001000500,17001000500,5,CT,1262517.0,0.0,"POLYGON ((-91.40340 39.95048, -91.39655 39.950...",0.665,0.787,0.113,0.133,0.035,10.62,41008.0,2175.0,813.0,11.3,1455.0,165.0,640,541,982.120690,916.275862,3.775862,1.793103,1.793103,0.913793,14573.370370,2554.592593,27.0,17.0,100431.535912,53611.933702,181.0,59.0


In [120]:
df_final.drop(columns=['STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'NAME', 'LSAD', 'ALAND', 'AWATER'], inplace=True)
df_final.head()

,GEOID,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,households,ba,den_ba,n_ba,nhh_computer,nhh_broadband,max_dn,max_up,dn10,dn100,dn250,fiber_100u,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,17001000100,"POLYGON ((-91.37766 39.94160, -91.37759 39.946...",0.813,0.861,0.246,0.068,0.000,10.78,48088.0,4761.0,2202.0,24.6,3381.0,831.0,1896,1791,931.548571,915.160000,2.268571,1.502857,1.502857,0.920000,45622.325000,10624.075000,40.0,30.0,83500.869110,52515.041885,191.0,92.0
1,17001000201,"POLYGON ((-91.39646 39.95621, -91.39631 39.965...",0.879,0.940,0.266,0.078,0.016,10.71,44855.0,1985.0,889.0,26.6,1400.0,373.0,836,781,872.587302,796.936508,2.761905,1.603175,1.603175,0.809524,24662.416667,5445.083333,12.0,7.0,103008.682403,48455.686695,233.0,74.0
2,17001000202,"POLYGON ((-91.39370 39.94678, -91.37759 39.946...",0.863,0.933,0.221,0.080,0.008,10.83,50375.0,2376.0,970.0,22.1,1527.0,338.0,905,837,1000.000000,1000.000000,3.944444,2.018519,2.018519,1.018519,31500.333333,1856.000000,3.0,3.0,99521.493976,60237.301205,166.0,62.0
3,17001000400,"POLYGON ((-91.42005 39.95081, -91.41917 39.951...",0.691,0.780,0.094,0.182,0.006,10.31,30164.0,3422.0,1316.0,9.4,2536.0,238.0,1027,909,519.804124,198.982021,1.757732,0.680412,0.680412,0.190722,17929.500000,4026.285714,28.0,17.0,86031.362416,25337.281879,149.0,62.0
4,17001000500,"POLYGON ((-91.40340 39.95048, -91.39655 39.950...",0.665,0.787,0.113,0.133,0.035,10.62,41008.0,2175.0,813.0,11.3,1455.0,165.0,640,541,982.120690,916.275862,3.775862,1.793103,1.793103,0.913793,14573.370370,2554.592593,27.0,17.0,100431.535912,53611.933702,181.0,59.0


In [121]:
df_final.to_file('data/IL/il_final.json', driver='GeoJSON')

# <a id='9'>Florida Datasets</a>

<a href='#0'>Return to Top</a>

## Creating Florida Datasets

In [49]:
fl_tracts = us_tracts[us_tracts['STATEFP']=='12']
fl_tracts = fl_tracts.to_crs("EPSG:4326")
fl_tracts.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
11,12,095,016902,1400000US12095016902,12095016902,169.02,CT,9854753,501139,"POLYGON ((-81.44418 28.47207, -81.43981 28.472..."
12,12,095,016804,1400000US12095016804,12095016804,168.04,CT,16052327,165779,"POLYGON ((-81.37578 28.41113, -81.37269 28.421..."
21,12,011,110311,1400000US12011110311,12011110311,1103.11,CT,2366691,193908,"POLYGON ((-80.28031 26.02264, -80.27248 26.022..."
41,12,091,022001,1400000US12091022001,12091022001,220.01,CT,1744725,0,"POLYGON ((-86.63451 30.44949, -86.63219 30.449..."
58,12,011,060112,1400000US12011060112,12011060112,601.12,CT,2480149,142045,"POLYGON ((-80.26840 26.20835, -80.25773 26.208..."


In [146]:
# fixed_tiles_fl = gpd.sjoin(fixed_ookla_shp, fl_tracts, how='inner', op='intersects')
# fixed_fl = fixed_tiles_fl.reset_index(drop=True)
# fixed_fl.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,0320230131030330,154337,140734,7,82,15,"POLYGON ((-81.79321 26.32296, -81.78772 26.322...",33632,12,071,050602,1400000US12071050602,12071050602,506.02,CT,5866234,308413
1,0320230131030300,84137,111338,22,15,10,"POLYGON ((-81.80420 26.33281, -81.79871 26.332...",33632,12,071,050602,1400000US12071050602,12071050602,506.02,CT,5866234,308413
2,0320230131030303,126936,131065,8,69,12,"POLYGON ((-81.79871 26.32788, -81.79321 26.327...",33632,12,071,050602,1400000US12071050602,12071050602,506.02,CT,5866234,308413
3,0320230131030302,141614,167720,12,94,14,"POLYGON ((-81.80420 26.32788, -81.79871 26.327...",33632,12,071,050602,1400000US12071050602,12071050602,506.02,CT,5866234,308413
4,0320230131030332,86182,63513,10,42,15,"POLYGON ((-81.79321 26.31804, -81.78772 26.318...",33632,12,071,050602,1400000US12071050602,12071050602,506.02,CT,5866234,308413


In [147]:
fixed_fl.to_file('fixed_ookla_fl.json', driver='GeoJSON')

In [148]:
mobile_tiles_fl = gpd.sjoin(mobile_ookla_shp, fl_tracts, how='inner', op='intersects')
mobile_fl = mobile_tiles_fl.reset_index(drop=True)
mobile_fl.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER
0,0320212303112301,29214,10267,40,18,10,"POLYGON ((-82.32605 27.93618, -82.32056 27.936...",3167,12,057,013316,1400000US12057013316,12057013316,133.16,CT,2605110,237789
1,0320212303112310,27486,12339,29,17,8,"POLYGON ((-82.32056 27.93618, -82.31506 27.936...",3167,12,057,013316,1400000US12057013316,12057013316,133.16,CT,2605110,237789
2,0320212303112330,17092,9304,37,9,7,"POLYGON ((-82.32056 27.92647, -82.31506 27.926...",3167,12,057,013316,1400000US12057013316,12057013316,133.16,CT,2605110,237789
3,0320212303112231,19753,7732,41,6,6,"POLYGON ((-82.33704 27.92647, -82.33154 27.926...",3167,12,057,013316,1400000US12057013316,12057013316,133.16,CT,2605110,237789
4,0320212303112132,39750,11503,35,22,14,"POLYGON ((-82.32056 27.94103, -82.31506 27.941...",3167,12,057,013316,1400000US12057013316,12057013316,133.16,CT,2605110,237789


In [149]:
mobile_fl.to_file('mobile_ookla_fl.json', driver='GeoJSON')

In [59]:
def prepare_dataset(shapefile, acsfile, fccfile, FIPS):
      # import shapefile
      shpfile = gpd.read_file(shapefile)
      shpfile = shpfile.sort_values(by=['GEOID'])
      if len(shpfile.GEOID[0]) == 11 and int(FIPS) < 10:     
         shpfile['GEOID'] = shpfile['GEOID'].apply(lambda x: x[1:])
      
      # import ACS file
      data = pd.read_csv(acsfile)
      data = data.rename(columns={'geoid':'GEOID'})
      data.drop(columns=['state', 'county', 'tract', 'broadband', 'computer', 'black', 'hispanic', 'mhi.1', 'den_computers', 
         'n_black', 'n_hispanic', 'den_black', 'den_hispanic', 'state_lkp', 'den_age', 'n_computer', 'n_broadband', 
         'state_lkp', 'nhh_computer_any_internet', 'nhh_computer_and_dialup', 'nhh_computer_and_broadband',
         'nhh_computer_no_internet', 'nhh_no_computer', 'n_children', 'n_children_computer', 'n_children_computer_and_dialup',
         'n_children_computer_and_broadband', 'n_children_computer_no_internet', 'n_children_no_computer'], inplace=True)
      data[data < 0] = np.nan
      data['GEOID'] = data['GEOID'].apply(lambda x: str(x))
      if len(data.GEOID[0]) == 11 and int(FIPS) < 10:     
         data['GEOID'] = data['GEOID'].apply(lambda x: x[1:])
      
      # import FCC file
      data2 = pd.read_csv(fccfile)
      data2 = data2.rename(columns={'tract':'GEOID'})
      data2['GEOID'] = data2['GEOID'].apply(lambda x: str(x))
      data2.drop(columns=['state'], inplace=True)
      if len(data2.GEOID[0]) == 11 and int(FIPS) < 10:     
         data2['GEOID'] = data2['GEOID'].apply(lambda x: x[1:])
      
      # spatial join of Ookla files with census tract files
      tracts = us_tracts.loc[us_tracts['STATEFP']==FIPS].to_crs(4326).reset_index(drop=True)
      fixed_tiles = gpd.sjoin(fixed_ookla_shp, tracts, how='inner', op='intersects')
      fixed = fixed_tiles.reset_index(drop=True)
      mobile_tiles = gpd.sjoin(mobile_ookla_shp, tracts, how='inner', op='intersects')
      mobile = mobile_tiles.reset_index(drop=True)
      # Find weighted averages by tests
      wm = lambda x: np.average(x, weights = mobile.loc[x.index, 'tests'])
      mobile_agg = mobile.groupby(['GEOID'], as_index=False).agg(
                                    m_d_kbps_wm=('avg_d_kbps', wm),
                                    m_u_kbps_wm=('avg_u_kbps', wm),
                                    m_tests=('tests', sum),
                                    m_devices=('devices', sum))
      wm = lambda x: np.average(x, weights=fixed.loc[x.index, 'tests'])
      fixed_agg = fixed.groupby(['GEOID'], as_index=False).agg(
                                    f_d_kbps_wm=('avg_d_kbps', wm),
                                    f_u_kbps_wm = ('avg_u_kbps', wm),
                                    f_tests = ('tests', sum),
                                    f_devices = ('devices', sum))
      fixed_agg['GEOID'] = fixed_agg['GEOID'].apply(lambda x: int(x))
      mobile_agg['GEOID'] = mobile_agg['GEOID'].apply(lambda x: int(x))
      data3 = mobile_agg.merge(fixed_agg, how='outer', on='GEOID')
      data3['GEOID'] = data3['GEOID'].apply(lambda x: str(x))
      if len(data3.GEOID[0]) == 11 and int(FIPS) < 10:     
         data3['GEOID'] = data3['GEOID'].apply(lambda x: x[1:])


      # Join spatial file with ACS, FCC, and Ookla dataframe
      df_1 = shpfile.merge(data, how="left", on="GEOID")
      df_2 = df_1.merge(data2, how="left", on="GEOID")
      df_final = df_2.merge(data3, how="left", on="GEOID")
      df_final.drop(columns=['STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'NAME', 'LSAD', 'ALAND', 'AWATER'], inplace=True)
      return df_final


In [60]:
shapefile = "data/FL/fl_spdf/fl_spdf.shp"
acsfile = 'data/FL/acs_2019_FL.csv'
fccfile = 'data/FL/fcc_477_census_tract_FL.csv'
FIPS='12'

In [61]:
fl_df = prepare_dataset(shapefile, acsfile, fccfile, FIPS)
fl_df.head()

,GEOID,geometry,f_broadband,f_computer,f_ba,f_black,f_hispanic,log_mhi,mhi,population,households,ba,den_ba,n_ba,nhh_computer,nhh_broadband,max_dn,max_up,dn10,dn100,dn250,fiber_100u,m_d_kbps_wm,m_u_kbps_wm,m_tests,m_devices,f_d_kbps_wm,f_u_kbps_wm,f_tests,f_devices
0,12001000200,"POLYGON ((-82.33935 29.64490, -82.33937 29.648...",0.816,0.968,0.279,0.184,0.130,9.79,17786.0,7557.0,2583.0,27.9,1876.0,524.0,2501,2109,979.959184,53.995265,1.925170,1.000000,1.000000,0.020408,43673.842105,10471.221053,95.0,49.0,71801.364312,39138.881041,269.0,127.0
1,12001000301,"POLYGON ((-82.33922 29.66668, -82.33919 29.673...",0.723,0.879,0.450,0.429,0.069,10.40,33011.0,4426.0,1750.0,45.0,2717.0,1222.0,1539,1265,990.015000,34.653840,1.560000,1.060000,1.000000,0.000000,35426.132075,6211.150943,53.0,41.0,79226.713542,14600.953125,192.0,73.0
2,12001000302,"POLYGON ((-82.33916 29.67913, -82.33909 29.688...",0.818,0.885,0.315,0.223,0.241,10.40,32750.0,2319.0,1084.0,31.5,1710.0,539.0,959,887,989.663158,44.797558,1.178947,1.010526,1.000000,0.010526,44956.519231,8486.615385,52.0,38.0,79433.879310,11815.620690,116.0,57.0
3,12001000400,"POLYGON ((-82.32413 29.66919, -82.32156 29.673...",0.786,0.849,0.245,0.560,0.078,10.59,39864.0,6256.0,2151.0,24.5,4068.0,995.0,1827,1690,947.382979,33.170979,1.329787,0.946809,0.946809,0.000000,50140.673469,8001.081633,49.0,36.0,85869.315534,26905.077670,206.0,64.0
4,12001000500,"POLYGON ((-82.33082 29.65338, -82.32659 29.653...",0.852,0.942,0.557,0.199,0.094,10.61,40588.0,4973.0,2522.0,55.7,3572.0,1989.0,2376,2149,967.066914,392.588372,2.130112,1.405204,1.364312,0.371747,53066.836957,11395.032609,92.0,50.0,71548.100694,25092.857639,288.0,109.0


In [62]:
fl_df.to_file('fl_final.json', driver='GeoJSON')